In [307]:
import sys

!{sys.executable} -m pip install setuptools
!{sys.executable} -m pip install python-dotenv
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install scikit-learn
!{sys.executable} -m pip install tensorflow
!{sys.executable} -m pip install mlflow
!{sys.executable} -m pip install dagshub
!{sys.executable} -m pip install keras-tuner
!{sys.executable} -m pip install scikeras


[notice] A new release of pip is available: 23.2.1 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [308]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import classification_report, confusion_matrix, make_scorer, f1_score
from scikeras.wrappers import KerasClassifier
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import mlflow
import mlflow.tensorflow
import dagshub
from scipy.stats import randint, loguniform
from tensorflow.keras.utils import plot_model
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder

In [309]:
# set up dagshub for mlflow tracking
dagshub.init(repo_owner='JonaKoenemann', repo_name='machine_failure_classification', mlflow=True)

Initialized MLflow to track repo "JonaKoenemann/machine_failure_classification"

Repository JonaKoenemann/machine_failure_classification initialized!

# Data preparation

In [310]:
data_path = "../../../data/predictive_maintenance.csv"

In [311]:
df = pd.read_csv(data_path)

In [312]:
categorical_features = ['Type']
numerical_features = df.select_dtypes(include=[np.number]).columns.tolist()
numerical_features.remove('UDI')
numerical_features.remove('Target')

# Feature Engineering: Polynomial Features
poly = PolynomialFeatures(degree=2, include_bias=False)

preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline(steps=[
            ('scaler', StandardScaler()),
            ('poly', poly)
        ]), numerical_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])

In [313]:
# Split for features (X) und labels (y)
X = df[numerical_features + categorical_features] # select Features 
y = df["Failure Type"]

In [314]:
# label_encoder = LabelEncoder()
# y_train_encoded = label_encoder.fit_transform(y_train)
y_encoded = pd.get_dummies(y).values

In [315]:
# Split for test and train data
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

In [316]:
# Test preprocessor pipeline
X_train_transformed = preprocessor.fit_transform(X_train)
X_test_transformed = preprocessor.transform(X_test)

print(X_train_transformed.shape)
print(X_test_transformed.shape)

(8000, 23)
(2000, 23)


# Model creation

In [330]:
# # Define model creation function
# def create_model(input_dim):
#     model = Sequential([
#         Dense(6, input_dim=input_dim),
#         Dense(12, activation='relu'),
#         Dense(6, activation='softmax')
#     ])
#     model.compile(optimizer='Adam', loss='sparse_categorical_crossentropy', metrics=['Accuracy'])
#     return model
# Define the model for multi-class classification
def create_model(input_dim, optimizer='adam'):
    model = Sequential()
    model.add(Dense(6, input_dim=input_dim))
    model.add(Dense(6, activation='softmax'))  # 6 output units for 6 classes
    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [331]:
model = KerasClassifier(model=create_model, input_dim=X_train_transformed.shape[1], verbose=0)

In [332]:
# Define hyperparameters distribution
param_dist = {
    #'model__learning_rate':loguniform(0.001, 0.1),
    'model__batch_size': randint(16, 65),
    'model__epochs': randint(10, 31),
    'model__optimizer': ['SGD', 'Adam'],
}

In [333]:
# Define the pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', model)
])

# Train and test

In [334]:
# # Set up RandomizedSearchCV with F1 score as the metric
# n_iter_search = 2
# scorer = make_scorer(f1_score, average='weighted')
# randomized_search = RandomizedSearchCV(estimator=pipeline, param_distributions=param_dist, n_iter=n_iter_search, cv=3, n_jobs=-1, scoring=scorer)
randomized_search = RandomizedSearchCV(estimator=pipeline, param_distributions=param_dist, cv=2, n_iter=2, scoring='f1_weighted')

In [335]:
mlflow.set_experiment("neural_net_rs_feature_simplified")
mlflow.tensorflow.autolog()

In [336]:
with mlflow.start_run() as run:
    # Fit RandomizedSearchCV
    randomized_search.fit(X_train, y_train)

    # Get the best parameters
    best_params = randomized_search.best_params_
    mlflow.log_params(best_params)

    # Get the best model
    best_model = randomized_search.best_estimator_

    # predicition
    y_pred = best_model.predict(X_test)

    # evaluation
    weighted_f1 = f1_score(y_true = y_test, y_pred = y_pred, average="weighted")
    print("\nweighted f1 score:", weighted_f1)
    mlflow.log_metric('weighted_f1_score', weighted_f1)
    
    mlflow.end_run()

c:\Users\jkoenemann\GitHub\machine_failure_classification\venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


2024/07/02 21:11:43 WARNING mlflow.utils.checkpoint_utils: Checkpoint logging is skipped, because checkpoint 'save_best_only' config is True, it requires to compare the monitored metric value, but the provided monitored metric value is not available.
2024/07/02 21:11:43 WARNING mlflow.utils.checkpoint_utils: Checkpoint logging is skipped, because checkpoint 'save_best_only' config is True, it requires to compare the monitored metric value, but the provided monitored metric value is not available.
2024/07/02 21:11:43 WARNING mlflow.utils.checkpoint_utils: Checkpoint logging is skipped, because checkpoint 'save_best_only' config is True, it requires to compare the monitored metric value, but the provided monitored metric value is not available.
2024/07/02 21:11:43 WARNING mlflow.utils.checkpoint_utils: Checkpoint logging is skipped, because checkpoint 'save_best_only' config is True, it requires to compare the monitored metric value, but the provided monitored metric value is not availab

2024/07/02 21:12:05 WARNING mlflow.utils.checkpoint_utils: Checkpoint logging is skipped, because checkpoint 'save_best_only' config is True, it requires to compare the monitored metric value, but the provided monitored metric value is not available.
2024/07/02 21:12:05 WARNING mlflow.utils.checkpoint_utils: Checkpoint logging is skipped, because checkpoint 'save_best_only' config is True, it requires to compare the monitored metric value, but the provided monitored metric value is not available.
2024/07/02 21:12:05 WARNING mlflow.utils.checkpoint_utils: Checkpoint logging is skipped, because checkpoint 'save_best_only' config is True, it requires to compare the monitored metric value, but the provided monitored metric value is not available.
2024/07/02 21:12:05 WARNING mlflow.utils.checkpoint_utils: Checkpoint logging is skipped, because checkpoint 'save_best_only' config is True, it requires to compare the monitored metric value, but the provided monitored metric value is not availab


weighted f1 score: 0.9662564519879095


# Evaluation

In [337]:
error_names = [
 'Heat Failure',
 'No Failure',
 'Overstrain Failure',
 'Power Failure',
 'Random Failure',
 'Tool wear Failure'] # get names of errors in correct order for confusion matrix

## Classification report

In [338]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.43      0.20      0.27        15
           1       0.98      1.00      0.99      1935
           2       0.50      0.15      0.24        13
           3       0.86      0.60      0.71        20
           4       0.00      0.00      0.00         6
           5       0.00      0.00      0.00        11

   micro avg       0.97      0.97      0.97      2000
   macro avg       0.46      0.33      0.37      2000
weighted avg       0.96      0.97      0.97      2000
 samples avg       0.97      0.97      0.97      2000



c:\Users\jkoenemann\GitHub\machine_failure_classification\venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## Normalized confusion matrix

In [340]:
# y_true = np.argmax(y_test, axis=1)
# y_pred = np.argmax(y_pred, axis=1)

conf_matrix_normalized = confusion_matrix(y_test, y_pred, normalize='true')

plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix_normalized, annot=True, cmap="Blues", cbar=False, xticklabels=error_names, yticklabels=error_names)
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.title("Normalized Confusion Matrix")
plt.show()

ValueError: Classification metrics can't handle a mix of multilabel-indicator and multiclass targets

## Train and validation loss

In [ ]:
history = best_model.model_.history
plt.figure(figsize=(8, 6))
plt.plot(history['loss'], label='Training Loss')
plt.plot(history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()